In [ ]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf


label_dict = {
    0: "",
    1: "",
    2: "",
    3: "",
    4: "",
    5: "",
}
'''
count = 0
file = open('labelmap.txt')
for line in file:
    label_dict[count]=line
    count += 1
print("hello",label_dict)'''
# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(12, 3), dtype=np.uint8)

def preprocess_image(image, input_size):
  """Preprocess the input image to feed to the TFLite model"""
#   img = tf.io.read_file(image_path)
#   img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(image, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
    """Returns a list of detection results, each a dictionary of object info."""

    signature_fn = interpreter.get_signature_runner()

    # Feed the input image to the model
    output = signature_fn(images=image)
    print(output)
    # Get all outputs from the model
    count = int(np.squeeze(output['output_0']))
    scores = np.squeeze(output['output_1'])
    classes = np.squeeze(output['output_2'])
    boxes = np.squeeze(output['output_3'])
    
    results = []
    for i in range(count):
        if scores[i] * 100 >= threshold:
            class_id = int(classes[i])  # Convert the class ID to an integer
            if class_id in label_dict:
                result = {
                    'bounding_box': boxes[i],
                    'class_id': class_id,
                    'score': scores[i]
                }
                print(result)
                results.append(result)

    return results


def run_odt_and_draw_results(image_path, interpreter, threshold=10):
    """Run object detection on the input image and draw the detection results"""
    # Load the input shape required by the model
    _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

    # Load the input image and preprocess it
    preprocessed_image, original_image = preprocess_image(
        image_path,
        (input_height, input_width)
        )

    # Run object detection on the input image
    results = detect_objects(interpreter, preprocessed_image, threshold=threshold)
    # Plot the detection results on the input image
    original_image_np = original_image.numpy().astype(np.uint8)
    for obj in results:
        # Convert the object bounding box from relative coordinates to absolute
        # coordinates based on the original image resolution
        ymin, xmin, ymax, xmax = obj['bounding_box']
        xmin = int(xmin * original_image_np.shape[1])
        xmax = int(xmax * original_image_np.shape[1])
        ymin = int(ymin * original_image_np.shape[0])
        ymax = int(ymax * original_image_np.shape[0])

        # Find the class index of the current object
        class_id = int(obj['class_id'])
        # Draw the bounding box and label on the image
        color = [int(c) for c in COLORS[class_id]]
        cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
        # Make adjustments to make the label visible for all objects
        y = ymin - 15 if ymin - 15 > 15 else ymin + 15
        label = "{}: {:.0f}%".format(label_dict[class_id], obj['score'] * 100)
        cv2.putText(original_image_np, label, (xmin, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Return the final image
    original_uint8 = original_image_np.astype(np.uint8)
    return original_uint8

def run_model(image,model,DETECTION_THRESHOLD):
    interpreter = tf.lite.Interpreter(model_path=model)
    interpreter.allocate_tensors()

    # Run inference and draw detection result on the local copy of the original file
    detection_result_image = run_odt_and_draw_results(
        image,
        interpreter,
        threshold=DETECTION_THRESHOLD
    )
    return detection_result_image

def showImage(image):        
    cv2.imshow('Object detection', image)

def testingModel(model):
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        detection_image = run_model(image,model,25)
        cv2.imwrite("test.jpg", detection_image)
        showImage(detection_image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
        # break
    cap.release()

testingModel('efficient3lite.tflite')